In [10]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip -O /tmp/horse-or-human.zip
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip  -O /tmp/validation-horse-or-human.zip

--2021-02-18 10:37:03--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 108.177.127.128, 2a00:1450:4013:c01::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  54.7MB/s    in 2.6s    

2021-02-18 10:37:06 (54.7 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2021-02-18 10:37:06--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 108.177.127.128, 2a00:1450:4013:c01::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting respons

In [12]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [16]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

train_horses_names = os.listdir('/tmp/horse-or-human/horses')
print(train_horses_names[:10])
train_human_names = os.listdir('/tmp/horse-or-human/humans')
print(train_human_names[:10])

validation_horse_names = os.listdir('/tmp/validation-horse-or-human/horses')
print(validation_horse_names[:10])

validation_human_names = os.listdir('/tmp/validation-horse-or-human/humans')
print(validation_human_names[:10])


['horse23-1.png', 'horse35-9.png', 'horse43-5.png', 'horse49-9.png', 'horse08-5.png', 'horse33-0.png', 'horse35-2.png', 'horse10-1.png', 'horse17-9.png', 'horse10-3.png']
['human14-09.png', 'human13-28.png', 'human11-04.png', 'human05-16.png', 'human03-08.png', 'human09-06.png', 'human15-21.png', 'human12-01.png', 'human06-28.png', 'human05-09.png']
['horse5-402.png', 'horse1-335.png', 'horse4-541.png', 'horse1-241.png', 'horse1-105.png', 'horse5-303.png', 'horse4-188.png', 'horse1-539.png', 'horse4-548.png', 'horse5-504.png']
['valhuman04-09.png', 'valhuman05-21.png', 'valhuman03-17.png', 'valhuman03-13.png', 'valhuman01-17.png', 'valhuman03-00.png', 'valhuman02-22.png', 'valhuman02-10.png', 'valhuman01-01.png', 'valhuman02-08.png']


In [17]:
import tensorflow as tf

In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3,), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 33, 33, 64)        3

In [21]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    '/tmp/horse-or-human',
    target_size = (300, 300),
    batch_size =128,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(
    rescale= 1./255
)

validation_generator = validation_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human', target_size = (300,300), class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [25]:
history = model.fit(train_generator, steps_per_epoch =8, epochs = 15, verbose=1, validation_data=validation_generator)

Epoch 1/15
8/8 [==============================] - 92s 10s/step - loss: 6.5910 - acc: 0.5920 - val_loss: 0.6198 - val_acc: 0.5273
Epoch 2/15
8/8 [==============================] - 76s 9s/step - loss: 0.6623 - acc: 0.6268 - val_loss: 5.0976 - val_acc: 0.5000
Epoch 3/15
8/8 [==============================] - 77s 9s/step - loss: 1.7743 - acc: 0.6280 - val_loss: 1.0930 - val_acc: 0.5156
Epoch 4/15
8/8 [==============================] - 77s 9s/step - loss: 0.6826 - acc: 0.7395 - val_loss: 1.9687 - val_acc: 0.5625
Epoch 5/15
8/8 [==============================] - 77s 9s/step - loss: 0.4622 - acc: 0.7646 - val_loss: 0.3200 - val_acc: 0.8711
Epoch 6/15
8/8 [==============================] - 77s 10s/step - loss: 0.3516 - acc: 0.8242 - val_loss: 1.2949 - val_acc: 0.6484
Epoch 7/15
8/8 [==============================] - 77s 10s/step - loss: 0.3246 - acc: 0.8442 - val_loss: 0.4805 - val_acc: 0.8906
Epoch 8/15
8/8 [==============================] - 96s 12s/step - loss: 0.2351 - acc: 0.9088 - val_los